In [ ]:
from PIL import Image
import os
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches

### Modify to your own paths

In [ ]:
base_path = "/your/own/root_directory/"
imgs_path = list(sorted(os.listdir("/your/own/images/")))
annot_path = list(sorted(os.listdir("/your/own//annotations/")))

In [ ]:
def generate_box(obj):
    xmin = int(obj.find('xmin').text)
    ymin = int(obj.find('ymin').text)
    xmax = int(obj.find('xmax').text)
    ymax = int(obj.find('ymax').text)
    return [xmin, ymin, xmax, ymax]

def generate_label(obj):
    if obj.find('name').text == "with_mask":
        return 1
    elif obj.find('name').text == "without_mask":
        return 2
    return 3

def generate_target(file): 
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, 'xml')
        objects = soup.find_all('object')
        num_objs = len(objects)
        boxes = []
        labels = []
        for i in objects:
            boxes.append(generate_box(i))
            labels.append(generate_label(i))
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        return target

### Plot 30 images with annotations

In [ ]:
N = 30
for ip,ap in zip(imgs_path[:N], annot_path[:N]):
  imp = base_path+"images/"+ip
  anp = base_path+"annotations/"+ap
  target = generate_target(anp)
  img = Image.open(imp).convert("RGB")
  fig,ax = plt.subplots(1)
  ax.imshow(img)
  for (box,label) in zip(target["boxes"],target["labels"]):
        xmin, ymin, xmax, ymax = box
        if label==1:
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='b',facecolor='none')
            print("with_mask")
        elif label==2:
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='g',facecolor='none')
            print("without_mask")
        else:
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')
            print("mask_weared_incorrect")
        ax.add_patch(rect)
        ax.axis("off")
  print(ip)
  plt.show()


### Check data distribution

In [ ]:
result = {}
for ip,ap in zip(imgs_path, annot_path):
  anp = base_path+"annotations/"+ap
  target = generate_target(anp)
  label = str(list(set(target["labels"].cpu().numpy())))

  if label in result:
    result[label] +=1
  else:
    result[label] =1

In [ ]:
print(result)

{'[1, 2, 3]': 42,
 '[1, 2]': 180,
 '[1, 3]': 30,
 '[1]': 516,
 '[2, 3]': 4,
 '[2]': 60,
 '[3]': 21}